# 📊 삼성전자·SK하이닉스 주가 분석 실습 노트북

이 노트북은 **yfinance**로 가져온 최근 1년치 데이터를 사용해:

- 거래량 0 제거 및 결측치 보간  
- 5일·20일 이동평균 & 볼린저 밴드 계산  
- 거래량 상위 5일 & 평균 거래량  
- 일간 수익률 및 연간 변동성  
- 종목 간 수익률 상관관계 히트맵  
- 볼린저 밴드 시각화  
- 계산 지표를 포함한 Excel 파일 저장

을 순차적으로 실습합니다.


In [ ]:
# (최초 1회) 필요한 패키지 설치
# !pip install yfinance pandas numpy matplotlib seaborn openpyxl

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# 한글 폰트 및 음수 표시 설정 (macOS: 주석 처리)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False


In [ ]:
# 최근 1년 데이터 범위 설정
end = datetime.today()
start = end - timedelta(days=365)

tickers = {'삼성전자':'005930.KS', 'SK하이닉스':'000660.KS'}

raw_data = {}
for name, ticker in tickers.items():
    df = yf.download(ticker, start=start, end=end)
    df['종목'] = name
    raw_data[name] = df
    print(f"{name} 데이터 shape:", df.shape)

# 멀티인덱스로 결합
price_df = pd.concat(raw_data).reset_index().set_index(['종목','Date'])
price_df.head()


In [ ]:
# ① 거래량 0 제거 ② 결측치는 직전값으로 보간
cleaned = (
    price_df[price_df['Volume'] != 0]
    .groupby(level=0)
    .apply(lambda df: df.ffill())
)

print("정제 후 데이터 shape:", cleaned.shape)
cleaned.head()


In [ ]:
# 5일, 20일 이동평균 및 볼린저 밴드(20일 ±2σ) 추가
def add_indicators(df):
    df = df.copy()
    df['MA5'] = df['Close'].rolling(5).mean()
    df['MA20'] = df['Close'].rolling(20).mean()
    std20 = df['Close'].rolling(20).std()
    df['BB_upper'] = df['MA20'] + 2 * std20
    df['BB_lower'] = df['MA20'] - 2 * std20
    return df

ind_df = cleaned.groupby(level=0, group_keys=False).apply(add_indicators)
ind_df.head(10)


In [ ]:
# 종목별 거래량 상위 5일과 평균 거래량 계산
for name in tickers:
    sub = ind_df.loc[name]
    top5 = sub.nlargest(5, 'Volume')
    avg_vol = sub['Volume'].mean()
    print(f"◆ {name} 거래량 TOP 5")
    display(top5[['Close','Volume']])
    print(f"평균 거래량: {avg_vol:,.0f}\n")


In [ ]:
# 일간 수익률 계산 및 연간 변동성(252거래일 기준) 구하기
returns = ind_df['Close'].groupby(level=0).apply(lambda x: x.pct_change())
daily_ret = returns.unstack(level=0)
annual_vol = daily_ret.std() * np.sqrt(252)

print("연간 변동성:")
display(annual_vol)
daily_ret.head()


In [ ]:
# 일간 수익률 상관관계 히트맵
corr = daily_ret.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('일간 수익률 상관관계')
plt.show()


In [ ]:
# 볼린저 밴드 시각화
for name in tickers:
    sub = ind_df.loc[name]
    plt.figure(figsize=(10,4))
    plt.plot(sub.index, sub['Close'], label='Close')
    plt.plot(sub.index, sub['MA20'], label='MA20')
    plt.fill_between(sub.index, sub['BB_lower'], sub['BB_upper'], alpha=0.2, label='BB')
    plt.title(f"{name} Bollinger Bands")
    plt.legend()
    plt.show()


In [ ]:
# 계산 결과를 Excel 파일로 저장
with pd.ExcelWriter('samsung_hynix_analysis.xlsx', engine='openpyxl') as writer:
    for name in tickers:
        ind_df.loc[name].to_excel(writer, sheet_name=name)
    daily_ret.to_excel(writer, sheet_name='DailyReturns')
print("Excel 파일 저장 완료 → samsung_hynix_analysis.xlsx")
